In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import pickle as p
from sklearn import metrics
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM

Using TensorFlow backend.


In [0]:
X=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/test_data.csv')
y=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/test_label.csv')
scaler = Normalizer().fit(X)
x = scaler.transform(X)

In [0]:
lstm_output_size = 70
with open("/content/drive/My Drive/Colab Notebooks/Pickle Models/knn.pkl", "rb")as f:
    knn = p.load(f)
with open("/content/drive/My Drive/Colab Notebooks/Pickle Models/DT.pkl", "rb")as f:
    rf = p.load(f)
with open("/content/drive/My Drive/Colab Notebooks/Pickle Models/NB.pkl", "rb")as f:
    nb = p.load(f)
with open("/content/drive/My Drive/Colab Notebooks/Pickle Models/SGD.pkl", "rb")as f:
    sgd = p.load(f)
cnn = Sequential()
cnn.add(Convolution1D(64, 3, activation="relu", input_shape=(41, 1)))
cnn.add(Convolution1D(64, 3, activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Convolution1D(128, 3, activation="relu"))
cnn.add(Convolution1D(128, 3, activation="relu"))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(LSTM(lstm_output_size))
cnn.add(Dropout(0.1))
cnn.add(Dense(5, activation="softmax"))
cnn.load_weights("/content/drive/My Drive/Colab Notebooks/HDF5 Models/checkpoint-09.hdf5")
cnn.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", 
            metrics=['accuracy'])
print("Created models and loaded weights from file")

Created models and loaded weights from file


In [0]:
y_test = np.array(y)
X = np.array(x)
X_test = np.reshape(X, (X.shape[0],X.shape[1],1))

In [0]:
y_knn = knn.predict(x)
print("KNN Done")

KNN Done


In [0]:
y_rf = rf.predict(x)
print("RF Done")

RF Done


In [0]:
y_cnn = cnn.predict_classes(X_test)
print("CNN Done")

CNN Done


In [0]:
pred = np.empty([y_rf.size,1], dtype=int)
a=0
for i in y_knn:
    if i == y_cnn[a]:
        pred[a] = i
    else:
        pred[a] = y_rf[a]
    a += 1

In [0]:
print("Accuracy =", metrics.accuracy_score(y, pred))
print("Confusion Matrix =\n", metrics.confusion_matrix(y, pred))
print("Classification Report =\n", metrics.classification_report(y, pred))

Accuracy = 0.9783300646239913
Confusion Matrix =
 [[11718    14     6   465     0]
 [    7 44896     4     0     0]
 [   19    25   777     3     0]
 [  787     2     2  3465     0]
 [   13     0     0     1     2]]
Classification Report =
               precision    recall  f1-score   support

           0       0.93      0.96      0.95     12203
           1       1.00      1.00      1.00     44907
           2       0.98      0.94      0.96       824
           3       0.88      0.81      0.85      4256
           4       1.00      0.12      0.22        16

   micro avg       0.98      0.98      0.98     62206
   macro avg       0.96      0.77      0.80     62206
weighted avg       0.98      0.98      0.98     62206

